https://adventofcode.com/2023/day/23

In [111]:
from collections import deque, defaultdict
from functools import cache
from heapq import heappush, heappop
from itertools import pairwise

import networkx as nx

In [2]:
with open("data/23.txt") as fh:
    data = fh.read()

In [3]:
testdata = """\
#.#####################
#.......#########...###
#######.#########.#.###
###.....#.>.>.###.#.###
###v#####.#v#.###.#.###
###.>...#.#.#.....#...#
###v###.#.#.#########.#
###...#.#.#.......#...#
#####.#.#.#######.#.###
#.....#.#.#.......#...#
#.#####.#.#.#########v#
#.#...#...#...###...>.#
#.#.#v#######v###.###v#
#...#.>.#...>.>.#.###.#
#####v#.#.###v#.#.###.#
#.....#...#...#.#.#...#
#.#########.###.#.#.###
#...###...#...#...#.###
###.###.#.###v#####v###
#...#...#.#.>.>.#.>.###
#.###.###.#.###.#.#v###
#.....###...###...#...#
#####################.#
"""

In [4]:
def parse_puzzle(puzzle):
    D = {}
    for r, line in enumerate(puzzle.splitlines()):
        for c, char in enumerate(line):
            if char != "#":
                D[c - r * 1j] = char
    return D

In [5]:
trailmap = parse_puzzle(testdata)

In [6]:
trailmap

{(1+0j): '.',
 (1-1j): '.',
 (2-1j): '.',
 (3-1j): '.',
 (4-1j): '.',
 (5-1j): '.',
 (6-1j): '.',
 (7-1j): '.',
 (17-1j): '.',
 (18-1j): '.',
 (19-1j): '.',
 (7-2j): '.',
 (17-2j): '.',
 (19-2j): '.',
 (3-3j): '.',
 (4-3j): '.',
 (5-3j): '.',
 (6-3j): '.',
 (7-3j): '.',
 (9-3j): '.',
 (10-3j): '>',
 (11-3j): '.',
 (12-3j): '>',
 (13-3j): '.',
 (17-3j): '.',
 (19-3j): '.',
 (3-4j): 'v',
 (9-4j): '.',
 (11-4j): 'v',
 (13-4j): '.',
 (17-4j): '.',
 (19-4j): '.',
 (3-5j): '.',
 (4-5j): '>',
 (5-5j): '.',
 (6-5j): '.',
 (7-5j): '.',
 (9-5j): '.',
 (11-5j): '.',
 (13-5j): '.',
 (14-5j): '.',
 (15-5j): '.',
 (16-5j): '.',
 (17-5j): '.',
 (19-5j): '.',
 (20-5j): '.',
 (21-5j): '.',
 (3-6j): 'v',
 (7-6j): '.',
 (9-6j): '.',
 (11-6j): '.',
 (21-6j): '.',
 (3-7j): '.',
 (4-7j): '.',
 (5-7j): '.',
 (7-7j): '.',
 (9-7j): '.',
 (11-7j): '.',
 (12-7j): '.',
 (13-7j): '.',
 (14-7j): '.',
 (15-7j): '.',
 (16-7j): '.',
 (17-7j): '.',
 (19-7j): '.',
 (20-7j): '.',
 (21-7j): '.',
 (5-8j): '.',
 (7-8j): '.'

In [7]:
def longest_walk_dfs(trailmap):
    drxn_lookup = {
        ">": [1],
        "^": [1j],
        "<": [-1],
        "v": [-1j],
        ".": [1, 1j, -1, -1j]
    }
    longest_walks = {}
    start, finish = start_finish(trailmap)
    stack = [(start, {start: 0})]
    while stack:
        pos, d = stack.pop()
        steps_so_far = d[pos]
        longest_walks[pos] = max(steps_so_far, longest_walks.get(pos, -1))
        for drxn in drxn_lookup[trailmap[pos]]:
            nabe = pos + drxn
            if nabe not in trailmap or nabe in d:
                continue
            newpos = nabe
            newd = d.copy()
            newd[newpos] = steps_so_far + 1
            stack.append((newpos, newd))
    
    return longest_walks[finish]


def start_finish(trailmap):
    ymax, ymin = -float("inf"), float("inf")
    start = finish = None
    for pos in trailmap:
        y = pos.imag
        if y > ymax:
            ymax = y
            start = pos
        if y < ymin:
            ymin = y
            finish = pos
    return start, finish    

In [8]:
longest_walk_dfs(parse_puzzle(testdata))

94

In [9]:
%%time
longest_walk_dfs(parse_puzzle(data))

CPU times: user 791 ms, sys: 0 ns, total: 791 ms
Wall time: 790 ms


2230

## Part 2

### Try contracting the graph

In [112]:
def contract(G, node, nabe):
    while G.degree(nabe) == 2:
        nextalong = [x for x in G.neighbors(nabe) if x != node][0]
        weightsum = G[node][nabe]["weight"] + G[nabe][nextalong]["weight"]
        if nextalong in G.neighbors(node):
            weightsum = max(weightsum, G[node][nextalong]["weight"])
        G.remove_edge(node, nabe)
        G.remove_edge(nabe, nextalong)
        G.add_edge(node, nextalong, weight=weightsum)
        nabe = nextalong
        
        
def total_edge_weight(G, pth):
    return sum(G[a][b]["weight"] for (a, b) in pairwise(pth))

In [124]:
%%time
trailmap = parse_puzzle(testdata)
start, finish = start_finish(trailmap)
G = nx.Graph()
G.add_nodes_from(trailmap)

for node in G:
    for drxn in [1, 1j, -1, -1j]:
        nabe = node + drxn
        if nabe in G:
            G.add_edge(node, nabe, weight=1)

junctions = {node for node in G if node in (start, finish) or G.degree(node) > 2}

for j in junctions:
    nabes = [x for x in G.neighbors(j) if G.degree(x) == 2]
    for nabe in nabes:
        try:
            contract(G, j, nabe)
        except KeyError:
            pass

maxsteps = 0
for pth in nx.all_simple_paths(G, start, finish):
    maxsteps = max(maxsteps, total_edge_weight(G, pth))
maxsteps

CPU times: user 2.57 ms, sys: 0 ns, total: 2.57 ms
Wall time: 2.58 ms


154

In [125]:
%%time
trailmap = parse_puzzle(data)
start, finish = start_finish(trailmap)
G = nx.Graph()
G.add_nodes_from(trailmap)

for node in G:
    for drxn in [1, 1j, -1, -1j]:
        nabe = node + drxn
        if nabe in G:
            G.add_edge(node, nabe, weight=1)

junctions = {node for node in G if node in (start, finish) or G.degree(node) > 2}

for j in junctions:
    nabes = [x for x in G.neighbors(j) if G.degree(x) == 2]
    for nabe in nabes:
        try:
            contract(G, j, nabe)
        except KeyError:
            pass

maxsteps = 0
for pth in nx.all_simple_paths(G, start, finish):
    maxsteps = max(maxsteps, total_edge_weight(G, pth))
maxsteps

CPU times: user 1min 7s, sys: 2.42 ms, total: 1min 7s
Wall time: 1min 7s


6542